In [ ]:
import sys
from pathlib import Path
import pandas as pd
import os

# Add project root to Python path
PROJECT_ROOT = Path.cwd().parent

if "notebooks" in Path.cwd().parts:
    os.chdir(PROJECT_ROOT)
    
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

# Import custom utility functions
from src.config.utils import (
    read_csv,
    aggregate_count,
    aggregate_sum,
    write_excel
)

from src.config.config import DATA_DIR, RESULTS_DIR, MOMENT_OF_SUICIDE_FEATURES, SOCIO_DEMOGRAPHIC_FEATURES

In [2]:
# Read CSV File
csv_file_path = DATA_DIR / "processed" / "encoded_data.csv"
df_encoded = read_csv(csv_file_path)
csv_file_path = DATA_DIR / "processed" / "group_set.csv"
df_groups = read_csv(csv_file_path)

In [3]:
FEATURES = list(set(SOCIO_DEMOGRAPHIC_FEATURES + MOMENT_OF_SUICIDE_FEATURES))
columns_to_aggregate = [
    column
    for column in df_encoded.columns
    if any(column.startswith(feature) for feature in FEATURES)
]
group_columns = [column for column in df_groups.columns if column.startswith("Group")]

In [4]:
df = df_encoded.merge(df_groups[["ID"] + group_columns], on="ID", how="left")

In [5]:
df = df[["DateY"] + group_columns + columns_to_aggregate]

In [14]:
df["DateY"] = df["DateY"].astype(int)

# Group Count

In [7]:
# Apply the function
count_result = aggregate_count(
    df,
    group_columns=["Group_A", "Group_AF", "Group_AG", "Group_AGF"],
    header="Count",
)
count_result = count_result.sort_index()
count_result

Count
Column    Value           
Group_A   00_18      10777
          19_34      39482
          35_64      62147
          65         14628
Group_AF  00_18_0     9435
          00_18_1     1342
          19_34_0    25729
          19_34_1    13753
          35_64_0    29480
          35_64_1    32667
          65_0        3494
          65_1       11134
Group_AG  00_18_F     7361
          00_18_M     3416
          19_34_F    10084
          19_34_M    29398
          35_64_F    13235
          35_64_M    48912
          65_F        3612
          65_M       11016
Group_AGF 00_18_F_0   6907
          00_18_F_1    454
          00_18_M_0   2528
          00_18_M_1    888
          19_34_F_0   8482
          19_34_F_1   1602
          19_34_M_0  17247
          19_34_M_1  12151
          35_64_F_0   8831
          35_64_F_1   4404
          35_64_M_0  20649
          35_64_M_1  28263
          65_F_0      1330
          65_F_1      2282
          65_M_0      2164
          65_M_1      8852

In [8]:
write_excel(
    file_path=RESULTS_DIR / "group_exploration.xlsx",
    data=count_result,
    sheet_name="count_result",
    mode="w",
    index=True,
)

# Feature Count

In [9]:
# Apply the function
feature_result = aggregate_sum(
    df,
    group_columns=["Group_A", "Group_AF", "Group_AG", "Group_AGF"],
    value_columns=columns_to_aggregate,
)
feature_result = feature_result.sort_index()
feature_result

Gender  Marital_Cohabitant  Marital_Cohabiting  \
Column                                                                
Group_A   00_18        3416                   9                   2   
          19_34       29398                2455                 289   
          35_64       48912                3365                 451   
          65          11016                 210                  27   
Group_AF  00_18_0      2528                   9                   2   
          00_18_1       888                   0                   0   
          19_34_0     17247                1688                 215   
          19_34_1     12151                 767                  74   
          35_64_0     20649                2002                 315   
          35_64_1     28263                1363                 136   
          65_0         2164                  59                  11   
          65_1         8852                 151                  16   
Group_AG  00_18_F         0                   4                   2   
          00_18_M      3416                   5                   0   
          19_34_F         0                 719                 123   
          19_34_M     29398                1736                 166   
          35_64_F         0                 951                 151   
          35_64_M     48912                2414                 300   
          65_F            0                  32                   4   
          65_M        11016                 178                  23   
Group_AGF 00_18_F_0       0                   4                   2   
          00_18_F_1       0                   0                   0   
          00_18_M_0    2528                   5                   0   
          00_18_M_1     888                   0                   0   
          19_34_F_0       0                 609                 110   
          19_34_F_1       0                 110                  13   
          19_34_M_0   17247                1079                 105   
          19_34_M_1   12151                 657                  61   
          35_64_F_0       0                 739                 129   
          35_64_F_1       0                 212                  22   
          35_64_M_0   20649                1263                 186   
          35_64_M_1   28263                1151                 114   
          65_F_0          0                  16                   2   
          65_F_1          0                  16                   2   
          65_M_0       2164                  43                   9   
          65_M_1       8852                 135                  14   

                     Marital_Divorced  Marital_Married  Marital_Separated  \
Column                                                                      
Group_A   00_18                     2               12                  0   
          19_34                  1109             6163                 68   
          35_64                  8117            32311                458   
          65                      946             7829                 50   
Group_AF  00_18_0                   0                9                  0   
          00_18_1                   2                3                  0   
          19_34_0                 696             4024                 39   
          19_34_1                 413             2139                 29   
          35_64_0                3588            15023                182   
          35_64_1                4529            17288                276   
          65_0                    213             1878                 11   
          65_1                    733             5951                 39   
Group_AG  00_18_F                   0                8                  0   
          00_18_M                   2                4                  0   
          19_34_F                 349             1778                 17   
          19_

In [10]:
write_excel(
    file_path=RESULTS_DIR / "group_exploration.xlsx",
    data=feature_result,
    sheet_name="feature_result",
    mode="a",
    index=True,
)

# Feature Percentage 

In [11]:
# Ensure the indices of both DataFrames match
if not feature_result.index.equals(count_result.index):
    raise ValueError("Indices of feature_result and count_result do not match.")

# Divide each column in feature_result by the "Count" column in count_result
percentage_feature_result = (
    feature_result.div(count_result["Count"], axis=0).mul(100).round(1)
)

percentage_feature_result


Gender  Marital_Cohabitant  Marital_Cohabiting  \
Column                                                                
Group_A   00_18        31.7                 0.1                 0.0   
          19_34        74.5                 6.2                 0.7   
          35_64        78.7                 5.4                 0.7   
          65           75.3                 1.4                 0.2   
Group_AF  00_18_0      26.8                 0.1                 0.0   
          00_18_1      66.2                 0.0                 0.0   
          19_34_0      67.0                 6.6                 0.8   
          19_34_1      88.4                 5.6                 0.5   
          35_64_0      70.0                 6.8                 1.1   
          35_64_1      86.5                 4.2                 0.4   
          65_0         61.9                 1.7                 0.3   
          65_1         79.5                 1.4                 0.1   
Group_AG  00_18_F       0.0                 0.1                 0.0   
          00_18_M     100.0                 0.1                 0.0   
          19_34_F       0.0                 7.1                 1.2   
          19_34_M     100.0                 5.9                 0.6   
          35_64_F       0.0                 7.2                 1.1   
          35_64_M     100.0                 4.9                 0.6   
          65_F          0.0                 0.9                 0.1   
          65_M        100.0                 1.6                 0.2   
Group_AGF 00_18_F_0     0.0                 0.1                 0.0   
          00_18_F_1     0.0                 0.0                 0.0   
          00_18_M_0   100.0                 0.2                 0.0   
          00_18_M_1   100.0                 0.0                 0.0   
          19_34_F_0     0.0                 7.2                 1.3   
          19_34_F_1     0.0                 6.9                 0.8   
          19_34_M_0   100.0                 6.3                 0.6   
          19_34_M_1   100.0                 5.4                 0.5   
          35_64_F_0     0.0                 8.4                 1.5   
          35_64_F_1     0.0                 4.8                 0.5   
          35_64_M_0   100.0                 6.1                 0.9   
          35_64_M_1   100.0                 4.1                 0.4   
          65_F_0        0.0                 1.2                 0.2   
          65_F_1        0.0                 0.7                 0.1   
          65_M_0      100.0                 2.0                 0.4   
          65_M_1      100.0                 1.5                 0.2   

                     Marital_Divorced  Marital_Married  Marital_Separated  \
Column                                                                      
Group_A   00_18                   0.0              0.1                0.0   
          19_34                   2.8             15.6                0.2   
          35_64                  13.1             52.0                0.7   
          65                      6.5             53.5                0.3   
Group_AF  00_18_0                 0.0              0.1                0.0   
          00_18_1                 0.1              0.2                0.0   
          19_34_0                 2.7             15.6                0.2   
          19_34_1                 3.0             15.6                0.2   
          35_64_0                12.2             51.0                0.6   
          35_64_1                13.9             52.9                0.8   
          65_0                    6.1             53.7                0.3   
          65_1                    6.6             53.4                0.4   
Group_AG  00_18_F                 0.0              0.1                0.0   
          00_18_M                 0.1              0.1                0.0   
          19_34_F                 3.5             17.6                0.2   
          19_

In [12]:
write_excel(
    file_path=RESULTS_DIR / "group_exploration.xlsx",
    data=percentage_feature_result,
    sheet_name="percentage_feature_result",
    mode="a",
    index=True,
)

# Group Count Years

In [15]:
# Apply the function
count_years_result = aggregate_count(
    df,
    group_columns=["Group_A", "Group_AF", "Group_AG", "Group_AGF"],
    value_columns=["DateY"],
)
count_years_result = count_years_result.sort_index()
count_years_result

2013  2014  2015  2016  2017  2018  2019  2020  2021  \
Column                                                                      
Group_A   00_18       356   442   481   475   730   772   950   843  1496   
          19_34      2265  3137  3200  3056  3605  3473  3772  3729  4342   
          35_64      4297  5115  5081  5011  5518  5616  5734  5960  6396   
          65          994  1146  1109  1215  1277  1299  1491  1474  1541   
Group_AF  00_18_0     209   315   362   372   614   675   852   736  1369   
          00_18_1     147   127   119   103   116    97    98   107   127   
          19_34_0     934  1724  1824  1812  2347  2302  2536  2513  3148   
          19_34_1    1331  1413  1376  1244  1258  1171  1236  1216  1194   
          35_64_0    1016  1723  1877  2024  2605  2718  2933  3207  3606   
          35_64_1    3281  3392  3204  2987  2913  2898  2801  2753  2790   
          65_0         98   151   185   214   296   289   381   392   470   
          65_1        896   995   924  1001   981  1010  1110  1082  1071   
Group_AG  00_18_F     154   221   239   278   473   516   637   538  1086   
          00_18_M     202   221   242   197   257   256   313   305   410   
          19_34_F     334   570   589   590   740   836   961   963  1315   
          19_34_M    1931  2567  2611  2466  2865  2637  2811  2766  3027   
          35_64_F     728   937   927   948  1097  1138  1223  1337  1464   
          35_64_M    3569  4178  4154  4063  4421  4478  4511  4623  4932   
          65_F        241   265   230   281   313   312   353   378   423   
          65_M        753   881   879   934   964   987  1138  1096  1118   
Group_AGF 00_18_F_0   118   188   206   249   434   485   604   497  1035   
          00_18_F_1    36    33    33    29    39    31    33    41    51   
          00_18_M_0    91   127   156   123   180   190   248   239   334   
          00_18_M_1   111    94    86    74    77    66    65    66    76   
          19_34_F_0   213   416   454   455   617   709   807   818  1164   
          19_34_F_1   121   154   135   135   123   127   154   145   151   
          19_34_M_0   721  1308  1370  1357  1730  1593  1729  1695  1984   
          19_34_M_1  1210  1259  1241  1109  1135  1044  1082  1071  1043   
          35_64_F_0   259   443   481   556   717   788   868   954  1103   
          35_64_F_1   469   494   446   392   380   350   355   383   361   
          35_64_M_0   757  1280  1396  1468  1888  1930  2065  2253  2503   
          35_64_M_1  2812  2898  2758  2595  2533  2548  2446  2370  2429   
          65_F_0       32    49    51    76   104   110   139   169   201   
          65_F_1      209   216   179   205   209   202   214   209   222   
          65_M_0       66   102   134   138   192   179   242   223   269   
          65_M_1      687   779   745   796   772   808   896   873   849   

                     2022  2023  
Column                           
Group_A   00_18      2093  2139  
          19_34      4359  4544  
          35_64      6488  6931  
          65         1570  1512  
Group_AF  00_18_0    1937  1994  
          00_18_1     156   145  
          19_34_0    3210  3379  
          19_34_1    1149  1165  
          35_64_0    3748  4023  
          35_64_1    2740  2908  
          65_0        514   504  
          65_1       1056  1008  
Group_AG  00_18_F    1586  1633  
          00_18_M     507   506  
          19_34_F    1498  1688  
          19_34_M    2861  2856  
          35_64_F    1627  1809  
          35_64_M    4861  5122  
          65_F        425   391  
          65_M       1145  1121  
Group_AGF 00_18_F_0  1525  1566  
          00_18_F_1    61    67  
          00_18_M_0   412   428  
          00_18_M_1    95    78  
          19_34_F_0  1329  1500  
          19_34_F_1   169   188  
          19_34_M_0  1881  1879  
          19_34_M_1   980   977  
          35_64_F_0  1233  1429  
          35_64_F_1   394   380  
      

In [21]:
write_excel(
    file_path=RESULTS_DIR / "group_exploration.xlsx",
    data=count_years_result,
    sheet_name="count_years_result",
    mode="a",
    index=True,
)

In [20]:
# Calculate total sum for each group (index 'Column') and year (columns in count_years_result)
group_year_totals = count_years_result.groupby("Column").transform("sum")

# Divide each value by the corresponding group-year total to calculate percentages
percentage_years_result = count_years_result.div(group_year_totals) * 100
percentage_years_result = percentage_years_result.round(1)  # Round to 1 decimal place

percentage_years_result

2013  2014  2015  2016  2017  2018  2019  2020  2021  \
Column                                                                      
Group_A   00_18       4.5   4.5   4.9   4.9   6.6   6.9   8.0   7.0  10.9   
          19_34      28.6  31.9  32.4  31.3  32.4  31.1  31.6  31.1  31.5   
          35_64      54.3  52.0  51.5  51.4  49.6  50.3  48.0  49.6  46.4   
          65         12.6  11.6  11.2  12.5  11.5  11.6  12.5  12.3  11.2   
Group_AF  00_18_0     2.6   3.2   3.7   3.8   5.5   6.0   7.1   6.1   9.9   
          00_18_1     1.9   1.3   1.2   1.1   1.0   0.9   0.8   0.9   0.9   
          19_34_0    11.8  17.5  18.5  18.6  21.1  20.6  21.2  20.9  22.9   
          19_34_1    16.8  14.4  13.9  12.7  11.3  10.5  10.3  10.1   8.7   
          35_64_0    12.8  17.5  19.0  20.7  23.4  24.4  24.6  26.7  26.2   
          35_64_1    41.5  34.5  32.5  30.6  26.2  26.0  23.4  22.9  20.3   
          65_0        1.2   1.5   1.9   2.2   2.7   2.6   3.2   3.3   3.4   
          65_1       11.3  10.1   9.4  10.3   8.8   9.1   9.3   9.0   7.8   
Group_AG  00_18_F     1.9   2.2   2.4   2.8   4.2   4.6   5.3   4.5   7.9   
          00_18_M     2.6   2.2   2.5   2.0   2.3   2.3   2.6   2.5   3.0   
          19_34_F     4.2   5.8   6.0   6.0   6.6   7.5   8.0   8.0   9.5   
          19_34_M    24.4  26.1  26.5  25.3  25.7  23.6  23.5  23.0  22.0   
          35_64_F     9.2   9.5   9.4   9.7   9.9  10.2  10.2  11.1  10.6   
          35_64_M    45.1  42.5  42.1  41.6  39.7  40.1  37.8  38.5  35.8   
          65_F        3.0   2.7   2.3   2.9   2.8   2.8   3.0   3.1   3.1   
          65_M        9.5   9.0   8.9   9.6   8.7   8.8   9.5   9.1   8.1   
Group_AGF 00_18_F_0   1.5   1.9   2.1   2.6   3.9   4.3   5.1   4.1   7.5   
          00_18_F_1   0.5   0.3   0.3   0.3   0.4   0.3   0.3   0.3   0.4   
          00_18_M_0   1.2   1.3   1.6   1.3   1.6   1.7   2.1   2.0   2.4   
          00_18_M_1   1.4   1.0   0.9   0.8   0.7   0.6   0.5   0.5   0.6   
          19_34_F_0   2.7   4.2   4.6   4.7   5.5   6.4   6.8   6.8   8.5   
          19_34_F_1   1.5   1.6   1.4   1.4   1.1   1.1   1.3   1.2   1.1   
          19_34_M_0   9.1  13.3  13.9  13.9  15.5  14.3  14.5  14.1  14.4   
          19_34_M_1  15.3  12.8  12.6  11.4  10.2   9.4   9.1   8.9   7.6   
          35_64_F_0   3.3   4.5   4.9   5.7   6.4   7.1   7.3   7.9   8.0   
          35_64_F_1   5.9   5.0   4.5   4.0   3.4   3.1   3.0   3.2   2.6   
          35_64_M_0   9.6  13.0  14.1  15.0  17.0  17.3  17.3  18.8  18.2   
          35_64_M_1  35.5  29.5  27.9  26.6  22.8  22.8  20.5  19.7  17.6   
          65_F_0      0.4   0.5   0.5   0.8   0.9   1.0   1.2   1.4   1.5   
          65_F_1      2.6   2.2   1.8   2.1   1.9   1.8   1.8   1.7   1.6   
          65_M_0      0.8   1.0   1.4   1.4   1.7   1.6   2.0   1.9   2.0   
          65_M_1      8.7   7.9   7.5   8.2   6.9   7.2   7.5   7.3   6.2   

                     2022  2023  
Column                           
Group_A   00_18      14.4  14.1  
          19_34      30.0  30.0  
          35_64      44.7  45.8  
          65         10.8  10.0  
Group_AF  00_18_0    13.3  13.2  
          00_18_1     1.1   1.0  
          19_34_0    22.1  22.3  
          19_34_1     7.9   7.7  
          35_64_0    25.8  26.6  
          35_64_1    18.9  19.2  
          65_0        3.5   3.3  
          65_1        7.3   6.7  
Group_AG  00_18_F    10.9  10.8  
          00_18_M     3.5   3.3  
          19_34_F    10.3  11.2  
          19_34_M    19.7  18.9  
          35_64_F    11.2  12.0  
          35_64_M    33.5  33.9  
          65_F        2.9   2.6  
          65_M        7.9   7.4  
Group_AGF 00_18_F_0  10.5  10.4  
          00_18_F_1   0.4   0.4  
          00_18_M_0   2.8   2.8  
          00_18_M_1   0.7   0.5  
          19_34_F_0   9.2   9.9  
          19_34_F_1   1.2   1.2  
          19_34_M_0  13.0  12.4  
          19_34_M_1   6.8   6.5  
          35_64_F_0   8.5   9.4  
          35_64_F_1   2.7   2.5  
      

In [22]:
write_excel(
    file_path=RESULTS_DIR / "group_exploration.xlsx",
    data=percentage_years_result,
    sheet_name="percentage_years_result",
    mode="a",
    index=True,
)